# Step 1: Install necessary libraries
We will install `unsloth`, `torch`, and `bitsandbytes` for fine-tuning, and also `ollama` (if needed) to export models later.


In [1]:
!pip install unsloth bitsandbytes torch --upgrade --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.

# Step 2: Load a base model using Unsloth
We use Unsloth's `FastLanguageModel` to load a Llama 3 model.
We'll specify 4-bit quantization (to save memory) and prepare it for fine-tuning.



In [5]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit",
    max_seq_length = 4096,
    dtype = torch.float16,
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Step 3: Create dummy fine-tuning data
We create a small dummy dataset of prompts and completions for fast fine-tuning.
You can replace this later with your real dataset.


In [6]:
from datasets import Dataset

# Create dummy training data
train_data = Dataset.from_dict({
    "input": [
        "What is the capital of France?",
        "Who wrote Hamlet?",
    ],
    "output": [
        "The capital of France is Paris.",
        "William Shakespeare wrote Hamlet.",
    ]
})

train_data = train_data.train_test_split(test_size=0.1)["train"]  # just training split


# Step 4: Prepare model for fine-tuning with LoRA
We enable LoRA to fine-tune efficiently with very few trainable parameters.
This keeps the model small and fast during training.


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,               # Rank
    lora_alpha = 16,     # Scaling
    lora_dropout = 0.05, # Regularization
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Important for LLaMA
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 42,
    use_rslora = True,     # Faster inference + less VRAM
    loftq_config = None,   # Not using loftq for now
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.1 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# Step 5: Set up the Trainer
We set training arguments like batch size, learning rate, and evaluation steps.
We'll use Hugging Face's `Trainer` to handle the training loop automatically.


In [15]:
from transformers import TrainingArguments, Trainer

# Tokenize the data
def formatting_prompts_func(example):
    prompt = f"### Question: {example['input']}\n### Answer: {example['output']}"
    tokenized = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=4096,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

train_data = train_data.map(formatting_prompts_func)

# Define training arguments
training_arguments = TrainingArguments(
    output_dir = "outputs",
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    optim = "adamw_torch", # Changed to AdamW from PyTorch
    save_steps = 10,
    logging_steps = 5,
    learning_rate = 2e-4,
    fp16 = True,  # Use fp16 instead of bf16
    max_grad_norm = 0.3,
    warmup_ratio = 0.03,
    lr_scheduler_type = "constant",
    disable_tqdm = False,
    report_to = [],   # no wandb
)

# Create the Trainer
trainer = Trainer(
    model = model,
    train_dataset = train_data,
    args = training_arguments,
)

# Start training
trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1 | Num Epochs = 1 | Total steps = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,636,096/1,000,000,000 (0.56% trained)


RuntimeError: PassManager::run failed

In [ ]:
from unsloth import FastLanguageModel

# Export the model to GGUF format
FastLanguageModel.export_to_ollama(
    model = model,
    tokenizer = tokenizer,
    model_path = "unsloth_model",  # Directory to save the model
    quantization = "q4_k_m",       # Quantization method
    model_name = "unsloth_model",  # Name for the Ollama model
    create_modelfile = True        # Automatically create Modelfile
)
